In [2]:
import pandas as pd
import numpy as np
import os
import subprocess
import shutil
import generator
import json
from pathlib import Path
import sklearn.cluster
from sklearn.metrics import f1_score

In [2]:
dirpath = Path('out')
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)

SAMPLEFILE = '6'
GERMAN_FILE_FORMATTING = 'true'
EPSILON = '0.5'
MINPTS = '4'
SPLITTING_SIZE = '16'
EVENLY_SIZED_SLICES = 'true'
DBSCAN = 'false'
p = subprocess.Popen(['java',
                      '-jar',
                      'SubScaleExtended.jar',
                      SAMPLEFILE,
                      GERMAN_FILE_FORMATTING,
                      EPSILON,
                      MINPTS,
                      SPLITTING_SIZE,
                      EVENLY_SIZED_SLICES,
                      DBSCAN],
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)
for line in p.stdout:
    print(str(line, "utf-8").replace('\r\n', ''))

Duration for creating the database: 72ms
Duration for calculating the density chunks: 11ms
Slice number: 8, size of collision map: 42
Slice number: 0, size of collision map: 2
Duration for calculating subspaces in slice 0: 13ms
Slice number: 1, size of collision map: 13
Duration for calculating subspaces in slice 1: 4ms
Slice number: 2, size of collision map: 7
Duration for calculating subspaces in slice 2: 2ms
Slice number: 3, size of collision map: 16
Duration for calculating subspaces in slice 3: 2ms
Slice number: 4, size of collision map: 16
Duration for calculating subspaces in slice 4: 2ms
Slice number: 5, size of collision map: 25
Duration for calculating subspaces in slice 5: 1ms
Slice number: 6, size of collision map: 28
Duration for calculating subspaces in slice 6: 2ms
Slice number: 7, size of collision map: 17
Duration for calculating subspaces in slice 7: 2ms
Slice number: 8, size of collision map: 18
Duration for calculating subspaces in slice 8: 2ms
Slice number: 9, size

In [1]:
DB, labels = generator.generate_subspacedata(100, 100, False, [[5, 4, 1, 0.001],
                                                              [5, 6, 1, 0.001],
                                                              [5, 9, 1, 0.001],
                                                              [5, 9, 1, 0.001],
                                                              [5, 9, 1, 0.001]])
np.savetxt("res/sample6.csv", DB, delimiter=",", fmt='%1.8f')

NameError: name 'generator' is not defined

In [5]:
# Potential subspaces will be searched for clusters using the dbscan algorithm

with open("out/results/clusters.csv", mode="w") as output_file:
    with os.scandir("out/results/subspaces") as it:
        for entry in it:
            if os.path.getsize(entry.path) > 0: # Check for empty file sometimes generated due to bug in SubScaleExtended.jar
                df = pd.read_csv(entry.path, header=None, delimiter="-")
                for _, row in df.iterrows():
                    points_indexes = json.loads(row[1])
                    dimensions = json.loads(row[0])
                    S = DB[points_indexes][:,dimensions]
                    clustering = sklearn.cluster.DBSCAN(eps=0.004).fit(S)

                    # Converting dbscan result in clusters.csv file-format
                    unique_labels = set(clustering.labels_)
                    for k in unique_labels:
                        U = np.array(points_indexes)[clustering.labels_ == k]
                        output_file.write(str(dimensions) + "-" + str(U) + "\n")

In [ ]:
# F1-Measure
pd.read_csv("out/results/clusters.csv", header=None, delimiter="-")
for _, row in df.iterrows():
    points_indexes = json.loads(row[1])
    dimensions = json.loads(row[0])
    S = DB[points_indexes][:,dimensions]



In [7]:
# https://www.ipd.kit.edu/~muellere/publications/BTW2011.pdf Erklärung zu f1 measure
labels

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])